# imports

In [77]:
from lxml import etree
from pathlib import Path
from pathlib import PurePosixPath
from datetime import datetime
import os
import csv 
import pandas as pd
import numpy as np
import pickle
import re
from pprint import pprint
import random
from itertools import chain

from ddp_util import chatomid_to_url, url_to_chatomid

import glob
import ddp_util

# including idx pickles

In [78]:
pickles = pd.read_fwf("../../data/in/sampling/pickles.txt", header=None)
pickles_git = pd.read_fwf("../../data/in/sampling/pickles_git.txt", header=None)

pickles = pickles.squeeze().rename("pickles")
pickles_git = pickles_git.squeeze().rename("pickles_git")

pickles_list = pickles.to_list()
pickles_list_git = pickles_git.to_list()

# load datasets, make clean dfs and lists

In [107]:
all_df_full = pd.read_json("../../data/in/sampling/charters_2022-09-18-2211.json")
all_df = all_df_full.explode("atom_id")["atom_id"].rename("atomid")
print(all_df)
all_list = all_df.to_list()

0         tag:www.monasterium.net,2011:/charter/069622fc...
1         tag:www.monasterium.net,2011:/charter/069622fc...
2         tag:www.monasterium.net,2011:/charter/069622fc...
3         tag:www.monasterium.net,2011:/charter/069622fc...
4         tag:www.monasterium.net,2011:/charter/069622fc...
                                ...                        
659165    tag:www.monasterium.net,2011:/charter/Wirtembe...
659166    tag:www.monasterium.net,2011:/charter/Wirtembe...
659167    tag:www.monasterium.net,2011:/charter/Wirtembe...
659168    tag:www.monasterium.net,2011:/charter/Wirtembe...
659169    tag:www.monasterium.net,2011:/charter/Wirtembe...
Name: atomid, Length: 659170, dtype: object


In [108]:
m_cv_df = pd.read_csv("../../data/in/sampling/1000charters_gsheet.csv", encoding="utf-8")
m_cv_df["atomid"] = m_cv_df["URL "].apply(lambda x : url_to_chatomid(x))
m_cv_df = m_cv_df["atomid"]
print(m_cv_df)
m_cv_list = m_cv_df.to_list()

0      tag:www.monasterium.net,2011:/charter/HU-MNL-D...
1      tag:www.monasterium.net,2011:/charter/DE-BayHS...
2      tag:www.monasterium.net,2011:/charter/AT-StiAS...
3      tag:www.monasterium.net,2011:/charter/HU-MNL-D...
4      tag:www.monasterium.net,2011:/charter/DE-HStAM...
                             ...                        
995    tag:www.monasterium.net,2011:/charter/AT-DOZA/...
996    tag:www.monasterium.net,2011:/charter/HU-MNL-D...
997    tag:www.monasterium.net,2011:/charter/DE-HStAM...
998    tag:www.monasterium.net,2011:/charter/DE-BayHS...
999    tag:www.monasterium.net,2011:/charter/AT-StiAG...
Name: atomid, Length: 1000, dtype: object


In [81]:
# load 1000 charter project sample from leech ?
#WSL ~/data/didip/tmp/data/leech/1000_Charters ... once copied, unclear where from
m_cv_leech_df = pd.read_fwf("../../data/in/sampling/atomid_list.txt", header=None)
m_cv_leech_df = m_cv_leech_df.squeeze().rename("atomid")
print(m_cv_leech_df)
m_cv_leech_list = m_cv_leech_df.to_list()

0      tag:www.monasterium.net,2011:/charter/AT-ADG/A...
1      tag:www.monasterium.net,2011:/charter/AT-ADG/A...
2      tag:www.monasterium.net,2011:/charter/AT-ADG/A...
3      tag:www.monasterium.net,2011:/charter/AT-ADG/A...
4      tag:www.monasterium.net,2011:/charter/AT-AES/U...
                             ...                        
995    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
996    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
997    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
998    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
999    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
Name: atomid, Length: 1000, dtype: object


In [82]:
# load 1000 charter project sample from other leechdump
#atzenhofer@21-PX010:/data/anguelos/monasterium/tmp/data/leech$ 
m_cv_leech_2nd_df = pd.read_fwf("../../data/in/sampling/atomid_list_second.txt", header=None)
m_cv_leech_2nd_df = m_cv_leech_2nd_df.squeeze().rename("atomid")
print(m_cv_leech_2nd_df)
m_cv_leech_2nd_list = m_cv_leech_2nd_df.to_list()

0      tag:www.monasterium.net,2011:/charter/AT-ADG/A...
1      tag:www.monasterium.net,2011:/charter/AT-ADG/A...
2      tag:www.monasterium.net,2011:/charter/AT-ADG/A...
3      tag:www.monasterium.net,2011:/charter/AT-ADG/A...
4      tag:www.monasterium.net,2011:/charter/AT-AES/U...
                             ...                        
995    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
996    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
997    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
998    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
999    tag:www.monasterium.net,2011:/charter/SK-SNA/4...
Name: atomid, Length: 1000, dtype: object


In [83]:
# load 1000 charter NLP sample, downloaded from Unicloud
m_nlp_df = pd.read_parquet("../../data/in/sampling/1000charters_nlp.parquet", engine="pyarrow")
m_nlp_df = m_nlp_df.explode("atom_id")["atom_id"].reset_index(drop=True)
print(m_nlp_df)
m_nlp_list = m_nlp_df.to_list()

0      tag:www.monasterium.net,2011:/charter/AFM/1.2.140
1      tag:www.monasterium.net,2011:/charter/StPCanRe...
2      tag:www.monasterium.net,2011:/charter/DEEDS/00...
3      tag:www.monasterium.net,2011:/charter/Bischoef...
4      tag:www.monasterium.net,2011:/charter/AFM/1.4....
                             ...                        
995    tag:www.monasterium.net,2011:/charter/Benedict...
996    tag:www.monasterium.net,2011:/charter/AFM/1.4....
997    tag:www.monasterium.net,2011:/charter/KlosterJ...
998    tag:www.monasterium.net,2011:/charter/Merkwuer...
999    tag:www.monasterium.net,2011:/charter/Lauresha...
Name: atom_id, Length: 1000, dtype: object


# make list of lists for contingency

In [84]:
lists_list = [m_nlp_list, m_cv_list, m_cv_leech_list, m_cv_leech_2nd_list]
#lists_list = [pickles_list, pickles_list_git]

# check differences

In [85]:
def get_var_name(variable):
    globals_dict = globals()
    return [var_name for var_name in globals_dict if globals_dict[var_name] is variable]

def isect(list1, list2):
    isect = list(set(list1).intersection(set(list2)))
    return isect

def diff(list1, list2):
    diff = list(set(list1).difference(set(list2)))
    return diff

In [86]:
tab = "\t"
nl = "\n"
sorted_list = sorted(lists_list)

for i in sorted_list:
    print(f"{(get_var_name(i))[0]} has a length of {len(i)}") #question to self: why is get-var-name inconsistent as to the order

print(nl)

for i in sorted_list:
    for j in sorted_list:
        intersection = isect(i, j)
        difference = diff(i, j)
        if i == j:
            break
        else:
            print(f"{(get_var_name(i))[0]} {tab} and {tab} {(get_var_name(j)[-2])} {tab} -> intersection of {len(intersection)} and a difference of {len(difference)}.")
        #   print(f"intersections: {intersection}")
        #   print(f"differences: {difference}")

m_nlp_list has a length of 1000
m_cv_leech_list has a length of 1000
m_cv_leech_2nd_list has a length of 1000
m_cv_list has a length of 1000


m_cv_leech_list 	 and 	 m_nlp_list 	 -> intersection of 0 and a difference of 989.
m_cv_leech_2nd_list 	 and 	 m_nlp_list 	 -> intersection of 0 and a difference of 989.
m_cv_leech_2nd_list 	 and 	 m_cv_leech_list 	 -> intersection of 989 and a difference of 0.
m_cv_list 	 and 	 m_nlp_list 	 -> intersection of 0 and a difference of 1000.
m_cv_list 	 and 	 m_cv_leech_list 	 -> intersection of 953 and a difference of 47.
m_cv_list 	 and 	 m_cv_leech_2nd_list 	 -> intersection of 953 and a difference of 47.


In [87]:
newlist = diff(m_cv_list, m_cv_leech_list)

In [88]:
len(newlist)

47

In [89]:
# from co-programming w anguelos: check 1000 charter dirs
#print(glob.glob("../../misc/1000_Charters/*/*/*/*json"))
# daniel_dirs = sorted(set(["/".join(f.split("/")[:-1]) for f in glob.glob("../../misc/1000_Charters/*/*/*/*.*.json")]))
# daniel_atoms = [open(f"{f}/url.txt").read() for f in daniel_dirs]
# daniel_atoms

#print(len(daniel_atoms))
# print(len(daniel_dirs))
# print(set(daniel_dirs)-set())

In [90]:
# print(chatomid_to_url("tag:www.monasterium.net,2011:/charter/AT-StaAR/UrkDominikanerkloster/StA_Retz%7CD%7CU1%7C1303"))
# print(chatomid_to_url("tag:www.monasterium.net,2011:/charter/AT-StaAR/UrkDominikanerkloster/StA_Retz%7CD"))

# "tag:www.monasterium.net,2011:/charter/SK-SNA/4156-SukromnyArchivBratislavskejKapituly/636"
# "tag:www.monasterium.net,2011:/charter/SK-SNA/4156-SukromnyArchivBratislavskejKapi"

#80 lengths cutoff?
# "tag:www.monasterium.net,2011:/charter/AT-NOeLA/HA_Seefeld-HardeggerUrk/Hardegger_"

# "tag:www.monasterium.net,2011:/charter/SK-SNA/4156-SukromnyArchivBratislavskejKapi"

## check set of sample atomids
conclusion: buggy urls -> buggy atomids -> uneven number

In [91]:
union_list = m_nlp_list + m_cv_list + m_cv_leech_list + m_cv_leech_2nd_list

In [92]:
print(len(union_list))
print(len(set(union_list)))

4000
2036


# reduce by previous nlp and cv sample

In [114]:
all_less_nlp = all_df[all_df.isin(m_nlp_list)]
#all_less_cv = all_df[ (  ~(all_df.isin(m_cv_list)))   ]

all_less_cv = all_df[ (  ~(all_df.isin(m_cv_list)))   ]
all_less_nlp_cv = all_df[~all_df.isin(m_nlp_list)][~all_df.isin(m_cv_list)]

In [155]:

all_df[~all_df.isin(m_nlp_list+m_cv_list)]
l_all_df=all_df.tolist()

import ddp_util
import random
import csv
assert len(set(m_nlp_list))==len(set(m_cv_list))
assert len(set(m_nlp_list+m_cv_list)) == 2000

eligible_3k=list(sorted(set(l_all_df)-set(m_nlp_list+m_cv_list)))
random.seed(1337)
random.shuffle(eligible_3k)
eligible_3k=sorted(eligible_3k[:3000])
with open("../../misc/3000_rndCharters/3k_urls.tsv","w") as f:
    w=csv.writer(f,delimiter="\t", dialect='unix', quoting=csv.QUOTE_NONE)
    for n, atomid in enumerate(eligible_3k):
        w.writerow([2000+n,atomid,ddp_util.chatomid_to_url(atomid=atomid)])






In [117]:
sorted(all_less_nlp.tolist()) == sorted(m_nlp_list)

True

# get charters with tenor

In [94]:

all_w_tenor

0        tag:www.monasterium.net,2011:/charter/069622fc...
1        tag:www.monasterium.net,2011:/charter/672f41e1...
2        tag:www.monasterium.net,2011:/charter/672f41e1...
3        tag:www.monasterium.net,2011:/charter/672f41e1...
4        tag:www.monasterium.net,2011:/charter/672f41e1...
                               ...                        
72297    tag:www.monasterium.net,2011:/charter/Wirtembe...
72298    tag:www.monasterium.net,2011:/charter/Wirtembe...
72299    tag:www.monasterium.net,2011:/charter/Wirtembe...
72300    tag:www.monasterium.net,2011:/charter/Wirtembe...
72301    tag:www.monasterium.net,2011:/charter/Wirtembe...
Name: atom_id, Length: 72302, dtype: object

# deduce previous nlp sample from all tenors

In [95]:
all_w_tenor_reduced = all_w_tenor[~all_w_tenor.isin(m_nlp_list)][~all_w_tenor.isin(m_cv_list)]
all_w_tenor_reduced

0        tag:www.monasterium.net,2011:/charter/069622fc...
1        tag:www.monasterium.net,2011:/charter/672f41e1...
2        tag:www.monasterium.net,2011:/charter/672f41e1...
3        tag:www.monasterium.net,2011:/charter/672f41e1...
4        tag:www.monasterium.net,2011:/charter/672f41e1...
                               ...                        
72297    tag:www.monasterium.net,2011:/charter/Wirtembe...
72298    tag:www.monasterium.net,2011:/charter/Wirtembe...
72299    tag:www.monasterium.net,2011:/charter/Wirtembe...
72300    tag:www.monasterium.net,2011:/charter/Wirtembe...
72301    tag:www.monasterium.net,2011:/charter/Wirtembe...
Name: atom_id, Length: 71265, dtype: object

# generate new 1k nlp sample

In [96]:
new_nlp_sample = all_w_tenor_reduced.sample(n=1000, random_state=50)
new_nlp_sample

48351    tag:www.monasterium.net,2011:/charter/KlosterJ...
29133    tag:www.monasterium.net,2011:/charter/CSGVI/13...
6102     tag:www.monasterium.net,2011:/charter/AFM/1.7.647
11901    tag:www.monasterium.net,2011:/charter/AT-StiAK...
71673    tag:www.monasterium.net,2011:/charter/WhalleyA...
                               ...                        
23919    tag:www.monasterium.net,2011:/charter/CodexDip...
21294    tag:www.monasterium.net,2011:/charter/Chronico...
921      tag:www.monasterium.net,2011:/charter/AbteiSan...
70296    tag:www.monasterium.net,2011:/charter/UrkVonZo...
35172    tag:www.monasterium.net,2011:/charter/DEEDS/00...
Name: atom_id, Length: 1000, dtype: object

# deduce new nlp sample from all

In [97]:
all_less_nlp_cv_nlp2 = all_less_nlp_cv[~all_less_nlp_cv.isin(new_nlp_sample)].reset_index(drop=True)
all_less_nlp_cv_nlp2

0         tag:www.monasterium.net,2011:/charter/069622fc...
1         tag:www.monasterium.net,2011:/charter/069622fc...
2         tag:www.monasterium.net,2011:/charter/069622fc...
3         tag:www.monasterium.net,2011:/charter/069622fc...
4         tag:www.monasterium.net,2011:/charter/069622fc...
                                ...                        
656165    tag:www.monasterium.net,2011:/charter/Wirtembe...
656166    tag:www.monasterium.net,2011:/charter/Wirtembe...
656167    tag:www.monasterium.net,2011:/charter/Wirtembe...
656168    tag:www.monasterium.net,2011:/charter/Wirtembe...
656169    tag:www.monasterium.net,2011:/charter/Wirtembe...
Name: atomid, Length: 656170, dtype: object

# generate new 3k sample

In [98]:
sample_3k = all_less_nlp_cv_nlp2.sample(n=3000, random_state=50)
sample_3k

626651    tag:www.monasterium.net,2011:/charter/RIXIii/1...
652177    tag:www.monasterium.net,2011:/charter/Urkunden...
233008    tag:www.monasterium.net,2011:/charter/DE-BayHS...
501344    tag:www.monasterium.net,2011:/charter/Illumini...
349079    tag:www.monasterium.net,2011:/charter/HU-MNL-D...
                                ...                        
470024    tag:www.monasterium.net,2011:/charter/HU-MNL-D...
234764    tag:www.monasterium.net,2011:/charter/DE-BayHS...
23541     tag:www.monasterium.net,2011:/charter/AT-DOZA/...
253939    tag:www.monasterium.net,2011:/charter/DE-BayHS...
409778    tag:www.monasterium.net,2011:/charter/HU-MNL-D...
Name: atomid, Length: 3000, dtype: object

# deduce 3k sample

In [99]:
all_less_samples = all_less_nlp_cv_nlp2[~all_less_nlp_cv_nlp2.isin(sample_3k)].reset_index(drop=True)
all_less_samples

0         tag:www.monasterium.net,2011:/charter/069622fc...
1         tag:www.monasterium.net,2011:/charter/069622fc...
2         tag:www.monasterium.net,2011:/charter/069622fc...
3         tag:www.monasterium.net,2011:/charter/069622fc...
4         tag:www.monasterium.net,2011:/charter/069622fc...
                                ...                        
653165    tag:www.monasterium.net,2011:/charter/Wirtembe...
653166    tag:www.monasterium.net,2011:/charter/Wirtembe...
653167    tag:www.monasterium.net,2011:/charter/Wirtembe...
653168    tag:www.monasterium.net,2011:/charter/Wirtembe...
653169    tag:www.monasterium.net,2011:/charter/Wirtembe...
Name: atomid, Length: 653170, dtype: object

# check lengths

In [100]:
print(len(all_df))
print(len(all_less_nlp))
print(len(all_less_cv))
print(len(all_less_nlp_cv))
print(len(all_less_nlp_cv_nlp2))
print(len(all_less_samples))

659170
658170
658170
657170
656170
653170


# extend atomids of 1k charters by 3k charters; +list of new nlp

In [101]:
atomids_3k = sample_3k.to_list()
atomids_4k = list(chain(m_cv_list, sample_3k.to_list()))
atomids_1k_nlp_new = new_nlp_sample.to_list()

# urls

In [102]:
urls_3k = [chatomid_to_url(i) for i in atomids_3k]
urls_4k = [chatomid_to_url(i) for i in atomids_4k]
urls_1k_nlp_new = [chatomid_to_url(i) for i in atomids_1k_nlp_new]

In [103]:
# random.seed(a=50)
# for i in random.sample(urls_4k, 10):
#     print(i)
# random.getstate()

# export 

## urls

In [104]:
with open(f"../../data/out/samples/urls_3k.csv", "w", newline="") as file:
    writer = csv.writer(file, delimiter = "\n")
    writer.writerow(urls_3k)

with open(f"../../data/out/samples/urls_4k.csv", "w", newline="") as file:
    writer = csv.writer(file, delimiter = "\n")
    writer.writerow(urls_4k)

with open(f"../../data/out/samples/urls_1k_nlp_new.csv", "w", newline="") as file:
    writer = csv.writer(file, delimiter = "\n")
    writer.writerow(urls_1k_nlp_new)

## atomids

In [105]:
with open(f"../../data/out/samples/atomids_3k.csv", "w", newline="") as file:
    writer = csv.writer(file, delimiter = "\n")
    writer.writerow(atomids_3k)

with open(f"../../data/out/samples/atomids_4k.csv", "w", newline="") as file:
    writer = csv.writer(file, delimiter = "\n")
    writer.writerow(atomids_4k)

with open(f"../../data/out/samples/atomids_1k_nlp_new.csv", "w", newline="") as file:
    writer = csv.writer(file, delimiter = "\n")
    writer.writerow(atomids_1k_nlp_new)

# HU in samples
says:
In the cv 1k charters project, the amount of /HU- charters was 38.5%.

In the old nlp sample, it was 0.63%.

In the new nlp sample, it is 0.6%.

In the new 3k one, it is 25.33%. 

In [106]:
# check amount of /HU- in samples
n = "\n"
exploded = all_df_full.explode("atom_id")
df = exploded[exploded["atom_id"].isin(m_cv_list)]
m_cv_HU = df[df["atom_id"].astype(str).str.contains("/HU-")]

df = exploded[exploded["atom_id"].isin(m_nlp_list)]
m_nlp_HU_old = df[df["atom_id"].astype(str).str.contains("/HU-")]

df = exploded[exploded["atom_id"].isin(new_nlp_sample.to_list())]
m_nlp_HU_new = df[df["atom_id"].astype(str).str.contains("/HU-")]

df = exploded[exploded["atom_id"].isin(atomids_3k)]
sample_3k_HU = df[df["atom_id"].astype(str).str.contains("/HU-")]

# difference of HU % in samples
print(f"""In the cv 1k charters project, the amount of /HU- charters was {round((len(m_cv_HU))/1000*100,2)}%.
{n}In the old nlp sample, it was {round((len(m_nlp_HU_old))/3000*100,2)}%.
{n}In the new nlp sample, it is {round((len(m_nlp_HU_new))/3000*100,2)}%.
{n}In the new 3k one, it is {round((len(sample_3k_HU))/3000*100,2)}%. """)

In the cv 1k charters project, the amount of /HU- charters was 38.5%.

In the old nlp sample, it was 0.63%.

In the new nlp sample, it is 0.6%.

In the new 3k one, it is 25.33%. 
